# Assigment

Write your own heuristic to solve the Aircrew Scheduling problem, it can be based on a Genetic Algorithm or even in a Adaptive Large Neighborhood Search.

### **Aircrew Scheduling Problem (ASP)**

The **Aircrew Scheduling Problem (ASP)** involves assigning crews (pilots and flight attendants) to flights over a planning horizon (typically a week or month) while satisfying various constraints. The problem is similar to the **Nurse Scheduling Problem (NSP)**, but has additional complexities due to flight regulations, work hours, layovers, and crew pairings. The objective is to create feasible schedules that minimize cost and maximize crew satisfaction, while ensuring all flights are covered.

---

### **Key Components of the Problem**:

1. **Flights**: Each flight has a specific start time, end time, and requires a set of crew members (pilots, co-pilots, and attendants).
2. **Crew**: A pool of crew members who need to be assigned to flights. Crew members may have different qualifications (e.g., some are only qualified to pilot specific aircraft types).
3. **Crew Pairing**: A sequence of flights that a crew member will operate, starting and ending at a base.
4. **Layovers**: Crew members often need rest periods when switching between flights, especially after long-haul flights.
5. **Work Hours**: There are regulations on the maximum number of consecutive hours or shifts a crew member can work, as well as minimum rest periods between flights.
6. **Base Assignment**: Crew members typically operate out of a specific base (home city), and their schedule must allow them to return to this base periodically.

---

### **Objective**:

The objective of the ASP is to **minimize total cost** (which could include crew wages, overtime, and deadheading costs), **maximize crew satisfaction**, and **ensure flight coverage** while adhering to the constraints related to crew pairings, regulations, and operational feasibility.

---

### **Variables**:

- $x_{ijk} \in \{0, 1\}$: A binary decision variable where $x_{ijk} = 1$ if crew member $i$ is assigned to flight $j$ on day $k$, and 0 otherwise.
- $o_i$: Overtime hours for crew member $i$.
- $c_j$: Cost of assigning a crew member to flight $j$, which includes wage and potential deadheading costs.

---

### **Constraints**:

#### 1. **Flight Coverage**:
Each flight must be covered by the required number of crew members (pilots and attendants):
$$
\sum_{i \in C} x_{ijk} \geq r_j, \quad \forall j \in F, \forall k \in D
$$
where $r_j$ is the number of crew members required for flight $j$.

#### 2. **Work Hour Regulations**:
Crew members must not exceed the maximum allowable work hours per day or week:
$$
\sum_{j \in F} w_j x_{ijk} \leq H_i^{\text{max}}, \quad \forall i \in C, \forall k \in D
$$
where $w_j$ is the working time (flight duration + preparation) for flight $j$, and $H_i^{\text{max}}$ is the maximum allowable working time for crew member $i$.

#### 3. **Rest Time Between Flights**:
There must be sufficient rest time between consecutive flights for each crew member. If crew member $i$ works flight $j_1$ and flight $j_2$, and the time gap between the flights is less than the required rest time $R$, they cannot be assigned both flights:
$$
x_{ijk_1} + x_{ijk_2} \leq 1, \quad \forall i \in C, \forall (j_1, j_2) \in F^2 \text{ with } \Delta t_{j_1,j_2} < R
$$
where $\Delta t_{j_1,j_2}$ is the time between flight $j_1$'s landing and flight $j_2$'s takeoff.

#### 4. **Layovers and Deadheading**:
If a crew member finishes their assignment at a destination that is not their base, they may need to deadhead (travel without operating a flight) to return to base:
$$
\sum_{j \in F} x_{ijk} + d_i \leq 1, \quad \forall i \in C, \forall k \in D
$$
where $d_i$ is a binary variable indicating if crew member $i$ is deadheading.

#### 5. **Pairing Constraints**:
Crew pairings (e.g., pilots and co-pilots) must remain consistent across all flights they operate together. For each flight $j$, the assigned crew members should satisfy the pairing rules:
$$
x_{i_1 jk} = x_{i_2 jk}, \quad \forall i_1, i_2 \in P_j
$$
where $P_j$ is the set of crew members that must be paired together on flight $j$.

#### 6. **Base Return**:
Crew members must return to their home base after completing a set of flights (pairing). This can be enforced by requiring that their last assigned flight returns to their base:
$$
\text{End location of the last flight in pairing } = \text{Base of the crew member}
$$

---

### **Objective Function**:

The objective is typically to **minimize costs** and **maximize crew satisfaction**, which can be expressed as:
$$
\min \sum_{i \in C} \left( \sum_{j \in F} c_j x_{ijk} + o_i \right) - \lambda \sum_{i \in C} \text{SatisfactionScore}_i
$$
where:
- $c_j$: Cost associated with assigning crew to flight $j$ (including wages, deadheading, etc.).
- $o_i$: Overtime for crew member $i$, penalized to avoid excessive working hours.
- $\lambda$: A weighting parameter balancing between minimizing costs and maximizing satisfaction.


In [2]:
import random
import numpy as np

def generate_aircrew_instance(num_crew, num_flights, num_days, max_hours_per_day, rest_time):
    # Random flight times and durations
    flights = [{'start': random.randint(0, 24), 'end': random.randint(1, 24), 'duration': random.randint(1, 8)} for _ in range(num_flights)]
    
    # Random base assignments for crew members
    crew_bases = [random.choice(['Base1', 'Base2', 'Base3']) for _ in range(num_crew)]
    
    # Maximum working hours for each crew member
    max_work_hours = [random.randint(max_hours_per_day // 2, max_hours_per_day) for _ in range(num_crew)]
    
    # Crew qualifications (which flights they can operate)
    qualifications = np.random.randint(0, 2, size=(num_crew, num_flights))
    
    return {
        'flights': flights,
        'crew_bases': crew_bases,
        'max_work_hours': max_work_hours,
        'qualifications': qualifications,
        'num_crew': num_crew,
        'num_flights': num_flights,
        'num_days': num_days,
        'rest_time': rest_time
    }

# Example usage:

instance = generate_aircrew_instance(num_crew=10, num_flights=20, num_days=7, max_hours_per_day=10, rest_time=12)

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def solve_aircrew_scheduling(instance):
    flights = instance['flights']
    crew_bases = instance['crew_bases']
    max_work_hours = instance['max_work_hours']
    qualifications = instance['qualifications']
    num_crew = instance['num_crew']
    num_flights = instance['num_flights']
    num_days = instance['num_days']
    rest_time = instance['rest_time']
    
    # Create Gurobi model
    model = gp.Model('Aircrew Scheduling')
    
    # Decision variables x[i,j,k] = 1 if crew i is assigned to flight j on day k
    x = model.addVars(num_crew, num_flights, num_days, vtype=GRB.BINARY, name="x")
    
    # Overtime variables for each crew member
    o = model.addVars(num_crew, vtype=GRB.CONTINUOUS, name="o")
    
    # Objective: Minimize overtime and assignment costs (assuming flight cost as 1 per assignment)
    model.setObjective(
        gp.quicksum(o[i] for i in range(num_crew)) + 
        gp.quicksum(x[i, j, k] for i in range(num_crew) for j in range(num_flights) for k in range(num_days)),
        GRB.MINIMIZE
    )
    
    # Constraints
    
    # 1. Flight coverage: Each flight must be fully crewed on each day
    for j in range(num_flights):
        for k in range(num_days):
            model.addConstr(gp.quicksum(x[i, j, k] for i in range(num_crew)) >= 2, name=f"FlightCoverage_{j}_{k}")
    
    # 2. Maximum working hours per crew
    for i in range(num_crew):
        for k in range(num_days):
            total_hours = gp.quicksum(flights[j]['duration'] * x[i, j, k] for j in range(num_flights))
            model.addConstr(total_hours <= max_work_hours[i] + o[i], name=f"MaxWorkHours_{i}_{k}")
    
    # 3. Minimum rest time between consecutive flights
    for i in range(num_crew):
        for k in range(num_days - 1):
            for j1 in range(num_flights):
                for j2 in range(num_flights):
                    if flights[j2]['start'] - flights[j1]['end'] < rest_time:
                        model.addConstr(x[i, j1, k] + x[i, j2, k+1] <= 1, name=f"RestTime_{i}_{j1}_{j2}_{k}")
    
    # 4. Qualifications: Crew members can only be assigned to flights they are qualified for
    for i in range(num_crew):
        for j in range(num_flights):
            for k in range(num_days):
                if qualifications[i, j] == 0:
                    model.addConstr(x[i, j, k] == 0, name=f"Qualification_{i}_{j}_{k}")
    
    # 5. Base return constraint (optional, can be handled by end-of-duty sequence)
    # Implement if needed based on how the instance defines base assignments
    
    # Optimize the model
    model.optimize()
    
    # Extract the solution
    if model.status == GRB.OPTIMAL:
        schedule = np.zeros((num_crew, num_flights, num_days), dtype=int)
        for i in range(num_crew):
            for j in range(num_flights):
                for k in range(num_days):
                    if x[i, j, k].x > 0.5:
                        schedule[i, j, k] = 1
        return schedule
    else:
        print("No optimal solution found.")
        return None

# Example usage:
instance = generate_aircrew_instance(num_crew=20, num_flights=20, num_days=7, max_hours_per_day=10, rest_time=12)
schedule = solve_aircrew_scheduling(instance)

if schedule is not None:
    print("Optimal Schedule Found:")
    print(schedule)


In [4]:
import numpy as np
import random

# Fitness function: Evaluates the quality of a given solution
def fitness(chromosome, items, max_weight):
    total_value = 0
    total_weight = 0
    
    # Calculate total value and weight of selected items
    for i in range(len(chromosome)):
        if chromosome[i] == 1:  # If item is selected
            total_value += items[i]['value']
            total_weight += items[i]['weight']
    
    # If the total weight exceeds the capacity, we penalize the fitness
    if total_weight > max_weight:
        return 0  # Invalid solution
    else:
        return total_value  # Valid solution

# Crossover: One-point crossover
def crossover(parent1, parent2):
    length = len(parent1)
    crossover_point = random.randint(1, length - 1)
    
    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    
    return child1, child2

# Mutation: Randomly flip bits
def mutation(chromosome, mutation_rate=0.1):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # Flip the bit
    return chromosome

# Selection: Tournament selection
def tournament_selection(population, fitness_scores, k=3):
    selected = random.sample(range(len(population)), k)
    best = max(selected, key=lambda x: fitness_scores[x])
    return population[best]

# Genetic Algorithm for the Knapsack Problem
def genetic_algorithm(items, max_weight, population_size=100, generations=500, mutation_rate=0.1):
    num_items = len(items)
    
    # Initialize the population with random chromosomes
    population = [np.random.randint(0, 2, size=num_items) for _ in range(population_size)]
    print(population)
    
    best_solution = None
    best_fitness = float('-inf')
    
    for generation in range(generations):
        # Evaluate fitness of the population
        fitness_scores = [fitness(chromosome, items, max_weight) for chromosome in population]
        
        # Keep track of the best solution found so far
        for i in range(population_size):
            if fitness_scores[i] > best_fitness:
                best_fitness = fitness_scores[i]
                best_solution = population[i]
        
        # Create the next generation
        new_population = []
        
        while len(new_population) < population_size:
            # Selection
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            
            # Crossover
            child1, child2 = crossover(parent1, parent2)
            
            # Mutation
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)
            
            new_population.append(child1)
            new_population.append(child2)
        

        population = new_population[:population_size]
        
        print(f"Generation {generation}, Best Fitness: {best_fitness}")
    
    return best_solution, best_fitness

# Example usage

# Define the knapsack problem instance (items with value and weight)
items = [
    {'value': 10, 'weight': 5},
    {'value': 40, 'weight': 4},
    {'value': 30, 'weight': 6},
    {'value': 50, 'weight': 3},
    {'value': 35, 'weight': 7},
    {'value': 25, 'weight': 2}
]

# Maximum weight capacity of the knapsack
max_weight = 10

# Run the genetic algorithm
best_solution, best_fitness = genetic_algorithm(items, max_weight, population_size=100, generations=50, mutation_rate=0.1)

# Display the best solution
print("Best Solution (Items Selected):", best_solution)
print("Best Fitness (Total Value):", best_fitness)


[array([1, 1, 1, 1, 1, 0]), array([1, 1, 1, 1, 1, 0]), array([0, 0, 1, 0, 0, 0]), array([0, 0, 1, 0, 0, 1]), array([0, 1, 0, 1, 1, 1]), array([0, 1, 0, 0, 1, 1]), array([1, 0, 1, 0, 0, 1]), array([0, 1, 1, 1, 0, 0]), array([1, 1, 1, 1, 1, 1]), array([0, 1, 0, 1, 1, 0]), array([0, 0, 1, 1, 0, 0]), array([0, 0, 1, 0, 1, 0]), array([0, 1, 0, 0, 0, 0]), array([1, 1, 1, 1, 0, 1]), array([0, 0, 1, 0, 1, 0]), array([0, 1, 1, 1, 1, 0]), array([0, 0, 1, 0, 0, 0]), array([1, 0, 0, 0, 1, 1]), array([0, 1, 0, 1, 1, 1]), array([1, 0, 0, 0, 1, 0]), array([1, 1, 1, 1, 0, 0]), array([1, 0, 0, 0, 0, 0]), array([0, 1, 1, 0, 1, 1]), array([0, 0, 0, 1, 0, 1]), array([0, 0, 1, 1, 0, 1]), array([1, 1, 0, 0, 1, 1]), array([1, 0, 1, 1, 1, 0]), array([1, 1, 0, 0, 0, 0]), array([1, 0, 0, 0, 1, 0]), array([0, 1, 0, 0, 0, 0]), array([1, 0, 1, 0, 0, 0]), array([0, 0, 1, 0, 0, 1]), array([1, 1, 0, 0, 1, 1]), array([0, 1, 0, 1, 1, 0]), array([0, 1, 1, 1, 1, 0]), array([1, 0, 0, 1, 0, 1]), array([1, 0, 0, 0, 1, 0]), 

In [5]:
# Define a harder knapsack problem instance (items with value and weight)
items_harder = [
    {'value': 90, 'weight': 30},
    {'value': 70, 'weight': 20},
    {'value': 50, 'weight': 25},
    {'value': 60, 'weight': 35},
    {'value': 80, 'weight': 28},
    {'value': 100, 'weight': 40},
    {'value': 20, 'weight': 15},
    {'value': 40, 'weight': 10},
    {'value': 30, 'weight': 12},
    {'value': 110, 'weight': 50},
    {'value': 75, 'weight': 23},
    {'value': 85, 'weight': 33},
    {'value': 55, 'weight': 18},
    {'value': 95, 'weight': 27},
    {'value': 65, 'weight': 22},
    {'value': 125, 'weight': 55},
    {'value': 95, 'weight': 38},
    {'value': 85, 'weight': 30},
    {'value': 120, 'weight': 48},
    {'value': 140, 'weight': 60}
]

# Maximum weight capacity of the knapsack
max_weight_harder = 150

# Run the genetic algorithm on the harder problem instance
best_solution_harder, best_fitness_harder = genetic_algorithm(
    items_harder, 
    max_weight_harder, 
    population_size=100, 
    generations=100, 
    mutation_rate=0.1
)

# Display the best solution and fitness for the harder problem
print("Best Solution (Items Selected):", best_solution_harder)
print("Best Fitness (Total Value):", best_fitness_harder)


[array([1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0]), array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1]), array([0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]), array([1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]), array([0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1]), array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]), array([1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0]), array([0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]), array([1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]), array([0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]), array([1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1]), array([0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1]), array([1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]), array([1, 0, 1, 1, 0, 0, 1, 0, 1,